# Data exploration &  pré-processing

In [1]:
import pandas as pd
s1_df = pd.read_pickle("../src/model/data/S2/S2.pkl")
s1_activity = pd.read_csv("../src/model/data/S2/S2_activity.csv")

In [2]:
s1_keys = s1_df.keys()

L'id du sujet

In [3]:
s1_df['subject']

'S2'

Fréquence cardiaque grâce à l'électrocardiographie (ECG)

In [4]:
s1_df['rpeaks']

array([     33,     700,    1316, ..., 5742473, 5742909, 5743349],
      dtype=int32)

Contient toutes les données enregistrer par RespiBAN (torse) et Empatica E4 (poignet)

Chaque donnée est descrite dans le fichier info.txt de chaque dossier patient

In [15]:
s1_df['signal']

{'chest': {'ACC': array([[ 0.91139996,  0.03219998,  0.051     ],
         [ 0.91120005,  0.03240001,  0.04900002],
         [ 0.90900004,  0.03460002,  0.04579997],
         ...,
         [ 0.91659999,  0.02900004, -0.04180002],
         [ 0.91719997,  0.03120005, -0.04220003],
         [ 0.91620004,  0.03120005, -0.03920001]]),
  'ECG': array([[-0.02503967],
         [-0.02778625],
         [-0.03515625],
         ...,
         [ 0.06454468],
         [ 0.06646729],
         [ 0.07122803]]),
  'EMG': array([[-1.5],
         [-1.5],
         [-1.5],
         ...,
         [-1.5],
         [-1.5],
         [-1.5]]),
  'EDA': array([[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]),
  'Temp': array([[-273.15],
         [-273.15],
         [-273.15],
         ...,
         [-273.15],
         [-273.15],
         [-273.15]], dtype=float32),
  'Resp': array([[-3.0380249 ],
         [-3.04412842],
         [-3.07006836],
         ...,
         [

Moyenne de la fréquence cardiaque

In [4]:
s1_df['label']

array([70.9229657 , 71.29105528, 68.69383264, ..., 95.13817648,
       96.7405001 , 96.84438537])

In [5]:
s1_df['activity']

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

Information sur la personne

In [22]:
s1_df['questionnaire']

{'WEIGHT': 78.0,
 'Gender': ' m',
 'AGE': 34,
 'HEIGHT': 182.0,
 'SKIN': 3,
 'SPORT': 6}

In [ ]:
s1_activity

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.plot(s1_activity[' S2'], s1_activity['# SUBJECT_ID'], label="graph test")
ax.legend()
plt.show()

In [16]:
for signal in s1_df['signal']['chest']:
     print(signal)
    
#print(s1_df['signal']['wrist']['BVP'].shape)
#print(s1_activity[' S2'].shape)

ACC
ECG
EMG
EDA
Temp
Resp


Les signaux à analyser sont ACC (wrist & chest), ECG (chest), RESP (chest) ET BVP (wrist) 

In [ ]:
s1_df['signal']['wrist']['ACC'][0]

In [ ]:
merge_bvp = [signal[0] for signal in s1_df['signal']['wrist']['BVP']]
fig, ax = plt.subplots()
ax.plot(merge_bvp, label="graph test")
ax.legend()
plt.show()

In [3]:
def scaling_data(signal, frequence):
    scale_step = int(frequence / 4)
    
    scaled_signal = list()
    for i in range(0, len(signal), scale_step):
        somme_echelle_4_hz = sum(signal[i : i + scale_step]) / scale_step
        scaled_signal.append(somme_echelle_4_hz)
    return scaled_signal

"""
Supprime les signaux inutiles
on fait une copie pour garder les données du patient
"""
def remove_useless_data(patient):
    # on supprime les données inutiles
    copy_patient = patient
    del copy_patient['signal']['chest']['EDA']
    del copy_patient['signal']['chest']['EMG']
    del copy_patient['signal']['chest']['Temp']
    del copy_patient['signal']['wrist']['EDA']
    
    return copy_patient

"""
Retourne un dataframe traité provenant des données patients

:parameter:
patient (dict): donnée patient (fichier pkl)

:returns:
DataFrame: Données patient traitées
"""
def pretreatment(patient):
    patient = remove_useless_data(patient)
    # Données torses RespiBan : ACC ECG et Resp sont à 700Hz
    # On renome les clés pour y accéder plus facilement
    patient["signal_chest_ACC"] = scaling_data(patient["signal"]["chest"]["ACC"], 700)
    patient["signal_chest_ECG"] = scaling_data(patient["signal"]["chest"]["ECG"], 700)
    patient["signal_chest_Resp"] = scaling_data(patient["signal"]["chest"]["Resp"], 700)
    
    # On supprime les anciennes clés
    del patient['signal']['chest']['ACC']
    del patient["signal"]["chest"]["ECG"]
    del patient['signal']['chest']['Resp']

    # Données Poignet : ACC à 32Hz et BVP à 64Hz
    # Même traitement que pour les données torses
    patient["signal_wrist_ACC"] = scaling_data( patient["signal"]["wrist"]["ACC"], 32)
    patient["signal_wrist_BVP"] = scaling_data( patient["signal"]["wrist"]["BVP"], 64)
    
    del patient['signal']['wrist']['ACC']
    del patient['signal']['wrist']['BVP']

    # 0 pour homme, 1 pour femme
    patient['gender'] = 0  if patient['questionnaire']['Gender'] == 'm' else 1
    del patient ['questionnaire']['Gender']
    
    return pd.DataFrame.from_dict(patient)
    #return data['signal']

In [ ]:
s1_df['questionnaire']
#s1_test = pretreatment(s1_df)

